In [2]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf

from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import mlp4, cnn_lenet
from sklearn.impute import SimpleImputer

In [3]:
afq_dataset = AFQDataset(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md"],
    target_cols=["age"],
    index_col="subject_id",
)

In [4]:
afq_dataset.drop_target_na()

In [5]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1867
(1867, 4800)
(1867,)


In [6]:
# Here we impute missing bundles.
# We impute using the entire dataset, which permits data leakage between the train and test set.
# THIS IS BAD AND SHOULDN'T BE DONE IN PRODUCTION
# But we do it here to move straight to model training
# When we are more comfortabel with the models, we should come back
# here and train the imputer only on the training set.
imputer = SimpleImputer(strategy="median")
afq_dataset.X = imputer.fit_transform(afq_dataset.X)

In [8]:
# Separate into training, test, and validation sets
dataset_size = len(afq_dataset.subjects)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = int(0.15 * dataset_size)

full_dataset = afq_dataset.as_tensorflow_dataset()
full_dataset = full_dataset.shuffle(buffer_size=dataset_size, seed=0)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [9]:
batch_size = 128
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [10]:
# First let's check out the ``cnn_lenet`` from the tf_models module
model_cnn_lenet= cnn_lenet(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)

pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            870       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 26)

In [28]:
# Compile the model and fit it using training data
model_cnn_lenet.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cnn_lenet.fit(train_dataset, epochs=100, validation_data=val_dataset)

Epoch 1/100
11/11 [==============================] - 2s 43ms/step - loss: 13.8091 - mean_squared_error: 13.8091 - val_loss: 9.6470 - val_mean_squared_error: 9.6470
Epoch 2/100
11/11 [==============================] - 0s 27ms/step - loss: 14.4075 - mean_squared_error: 14.4075 - val_loss: 8.0002 - val_mean_squared_error: 8.0002
Epoch 3/100
11/11 [==============================] - 0s 27ms/step - loss: 13.2951 - mean_squared_error: 13.2951 - val_loss: 9.8860 - val_mean_squared_error: 9.8860
Epoch 4/100
11/11 [==============================] - 0s 26ms/step - loss: 14.6313 - mean_squared_error: 14.6313 - val_loss: 10.0691 - val_mean_squared_error: 10.0691
Epoch 5/100
11/11 [==============================] - 0s 26ms/step - loss: 13.9016 - mean_squared_error: 13.9016 - val_loss: 8.0365 - val_mean_squared_error: 8.0365
Epoch 6/100
11/11 [==============================] - 0s 26ms/step - loss: 13.2019 - mean_squared_error: 13.2019 - val_loss: 9.7383 - val_mean_squared_error: 9.7383
Epoch 7/100
11

11/11 [==============================] - 0s 26ms/step - loss: 10.7572 - mean_squared_error: 10.7572 - val_loss: 8.7999 - val_mean_squared_error: 8.7999
Epoch 51/100
11/11 [==============================] - 0s 25ms/step - loss: 11.5324 - mean_squared_error: 11.5324 - val_loss: 7.6402 - val_mean_squared_error: 7.6402
Epoch 52/100
11/11 [==============================] - 0s 25ms/step - loss: 12.2454 - mean_squared_error: 12.2454 - val_loss: 8.4177 - val_mean_squared_error: 8.4177
Epoch 53/100
11/11 [==============================] - 0s 25ms/step - loss: 10.4112 - mean_squared_error: 10.4112 - val_loss: 8.2535 - val_mean_squared_error: 8.2535
Epoch 54/100
11/11 [==============================] - 0s 25ms/step - loss: 10.8414 - mean_squared_error: 10.8414 - val_loss: 7.9911 - val_mean_squared_error: 7.9911
Epoch 55/100
11/11 [==============================] - 0s 25ms/step - loss: 11.1139 - mean_squared_error: 11.1139 - val_loss: 7.9014 - val_mean_squared_error: 7.9014
Epoch 56/100
11/11 [===

Epoch 100/100
11/11 [==============================] - 0s 25ms/step - loss: 9.7023 - mean_squared_error: 9.7023 - val_loss: 5.9706 - val_mean_squared_error: 5.9706


In [29]:
# Evaluate the model on the validation set
eval_cnn_lenet = model_cnn_lenet.evaluate(val_dataset)

3/3 [==============================] - 0s 6ms/step - loss: 7.4500 - mean_squared_error: 7.4500


We see that this model achieves a mean squared error of ~7.45, or an RMSE of ~2.5 years. This is probably an overly optimistic estimate because we imputed using the entire dataset.

In [31]:
# define an empty dictionary for results comparisons
results = {'cnn_lenet' : eval_cnn_lenet}

In [33]:
#``mlp4`` from the tf_models module
model_mlp4= mlp4(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_mlp4.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_mlp4.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_mlp4 = model_mlp4.evaluate(val_dataset)

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 100, 48)]         0         
                                                                 
 flatten_3 (Flatten)         (None, 4800)              0         
                                                                 
 dropout_9 (Dropout)         (None, 4800)              0         
                                                                 
 dense_17 (Dense)            (None, 500)               2400500   
                                                                 
 dropout_10 (Dropout)        (None, 500)               0         
                                                                 
 dense_18 (Dense)            (None, 500)               250500    
                                                                 
 dropout_11 (Dropout)        (None, 500)               0  

11/11 [==============================] - 0s 14ms/step - loss: 11.8574 - mean_squared_error: 11.8574
Epoch 59/100
11/11 [==============================] - 0s 15ms/step - loss: 9.0651 - mean_squared_error: 9.0651
Epoch 60/100
11/11 [==============================] - 0s 14ms/step - loss: 9.4175 - mean_squared_error: 9.4175
Epoch 61/100
11/11 [==============================] - 0s 15ms/step - loss: 9.4994 - mean_squared_error: 9.4994
Epoch 62/100
11/11 [==============================] - 0s 18ms/step - loss: 9.7253 - mean_squared_error: 9.7253
Epoch 63/100
11/11 [==============================] - 0s 14ms/step - loss: 9.4246 - mean_squared_error: 9.4246
Epoch 64/100
11/11 [==============================] - 0s 19ms/step - loss: 9.2009 - mean_squared_error: 9.2009
Epoch 65/100
11/11 [==============================] - 0s 15ms/step - loss: 8.5119 - mean_squared_error: 8.5119
Epoch 66/100
11/11 [==============================] - 0s 14ms/step - loss: 8.7386 - mean_squared_error: 8.7386
Epoch 67/100

In [34]:
# update the result
results.update({'mlp4': eval_mlp4})

In [17]:
from afqinsight.nn.tf_models import cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
#``cnn_vgg`` from the tf_models module
model_cnn_vgg= cnn_vgg(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_cnn_vgg.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cnn_vgg.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_cnn_vgg = model_cnn_vgg.evaluate(val_dataset)
# update the result
results.update({'cnn_vgg': eval_cnn_vgg})

pooling layers: 4
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 100, 64)           9280      
                                                                 
 conv1d_5 (Conv1D)           (None, 100, 64)           12352     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 50, 64)           0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 50, 128)           24704     
                                                                 
 conv1d_7 (Conv1D)           (None, 50, 128)           49280     
                                         

11/11 [==============================] - 5s 427ms/step - loss: 6.3439 - mean_squared_error: 6.3439
Epoch 43/100
11/11 [==============================] - 5s 427ms/step - loss: 6.1731 - mean_squared_error: 6.1731
Epoch 44/100
11/11 [==============================] - 5s 425ms/step - loss: 5.8094 - mean_squared_error: 5.8094
Epoch 45/100
11/11 [==============================] - 5s 424ms/step - loss: 5.9810 - mean_squared_error: 5.9810
Epoch 46/100
11/11 [==============================] - 5s 453ms/step - loss: 5.4627 - mean_squared_error: 5.4627
Epoch 47/100
11/11 [==============================] - 5s 466ms/step - loss: 5.7290 - mean_squared_error: 5.7290
Epoch 48/100
11/11 [==============================] - 5s 448ms/step - loss: 5.2090 - mean_squared_error: 5.2090
Epoch 49/100
11/11 [==============================] - 5s 438ms/step - loss: 4.6934 - mean_squared_error: 4.6934
Epoch 50/100
11/11 [==============================] - 5s 442ms/step - loss: 7.6627 - mean_squared_error: 7.6627
Epoch

In [35]:
# update the result
results.update({'cnn_vgg': eval_cnn_vgg})

original models = LSTM(100)

In [20]:
#``lstm1v0`` from the tf_models module
model_lstm1v0= lstm1v0(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile themodel and fit it using training data
model_lstm1v0.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm1v0.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_lstm1v0 = model_lstm1v0.evaluate(val_dataset)
# update the result
results.update({'lstm1v0': eval_lstm1v0})

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 lstm (LSTM)                 (None, 100)               59600     
                                                                 
 dense_10 (Dense)            (None, 1)                 101       
                                                                 
Total params: 59,701
Trainable params: 59,701
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 3s 95ms/step - loss: 87.0940 - mean_squared_error: 87.0940
Epoch 2/100
11/11 [==============================] - 1s 95ms/step - loss: 26.2610 - mean_squared_error: 26.2610
Epoch 3/100
11/11 [==============================] - 1s 94ms/step - loss: 12.9707 - mean_square

11/11 [==============================] - 1s 91ms/step - loss: 12.4412 - mean_squared_error: 12.4412
Epoch 67/100
11/11 [==============================] - 1s 90ms/step - loss: 12.8910 - mean_squared_error: 12.8910
Epoch 68/100
11/11 [==============================] - 1s 90ms/step - loss: 12.7092 - mean_squared_error: 12.7092
Epoch 69/100
11/11 [==============================] - 1s 90ms/step - loss: 12.6088 - mean_squared_error: 12.6088
Epoch 70/100
11/11 [==============================] - 1s 92ms/step - loss: 12.6559 - mean_squared_error: 12.6559
Epoch 71/100
11/11 [==============================] - 1s 91ms/step - loss: 12.9204 - mean_squared_error: 12.9204
Epoch 72/100
11/11 [==============================] - 1s 90ms/step - loss: 12.3465 - mean_squared_error: 12.3465
Epoch 73/100
11/11 [==============================] - 1s 90ms/step - loss: 12.8989 - mean_squared_error: 12.8989
Epoch 74/100
11/11 [==============================] - 1s 93ms/step - loss: 12.5435 - mean_squared_error: 12.5

In [21]:
#``lstm1`` from the tf_models module
model_lstm1= lstm1(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_lstm1.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm1.fit(train_dataset, epochs=100, validation_data=val_dataset)
# Evaluate the model on the validation set
eval_lstm1 = model_lstm1.evaluate(val_dataset)
# update the result
results.update({'lstm1': eval_lstm1})

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               59600     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 59,701
Trainable params: 59,701
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 4s 151ms/step - loss: 93.7480 - mean_squared_error: 93.7480 - val_loss: 44.1540 - val_mean_squared_error: 44.1540
Epoch 2/100
11/11 [==============================] - 2s 150ms/step - loss: 27.7673 - mean_squared_error: 27.7673 - val_loss: 17.2533 - val_mean_squared_

11/11 [==============================] - 1s 139ms/step - loss: 12.4733 - mean_squared_error: 12.4733 - val_loss: 12.8817 - val_mean_squared_error: 12.8817
Epoch 41/100
11/11 [==============================] - 2s 145ms/step - loss: 12.9039 - mean_squared_error: 12.9039 - val_loss: 14.0773 - val_mean_squared_error: 14.0773
Epoch 42/100
11/11 [==============================] - 1s 106ms/step - loss: 12.9990 - mean_squared_error: 12.9990 - val_loss: 12.5929 - val_mean_squared_error: 12.5929
Epoch 43/100
11/11 [==============================] - 1s 111ms/step - loss: 12.6990 - mean_squared_error: 12.6990 - val_loss: 13.0485 - val_mean_squared_error: 13.0485
Epoch 44/100
11/11 [==============================] - 1s 109ms/step - loss: 12.6336 - mean_squared_error: 12.6336 - val_loss: 14.0745 - val_mean_squared_error: 14.0745
Epoch 45/100
11/11 [==============================] - 1s 109ms/step - loss: 12.7978 - mean_squared_error: 12.7978 - val_loss: 14.5725 - val_mean_squared_error: 14.5725
Epoch

Epoch 89/100
11/11 [==============================] - 1s 102ms/step - loss: 12.8942 - mean_squared_error: 12.8942 - val_loss: 12.0384 - val_mean_squared_error: 12.0384
Epoch 90/100
11/11 [==============================] - 1s 102ms/step - loss: 13.0779 - mean_squared_error: 13.0779 - val_loss: 13.2093 - val_mean_squared_error: 13.2093
Epoch 91/100
11/11 [==============================] - 1s 103ms/step - loss: 12.4281 - mean_squared_error: 12.4281 - val_loss: 12.0951 - val_mean_squared_error: 12.0951
Epoch 92/100
11/11 [==============================] - 1s 103ms/step - loss: 12.4451 - mean_squared_error: 12.4451 - val_loss: 13.0885 - val_mean_squared_error: 13.0885
Epoch 93/100
11/11 [==============================] - 1s 102ms/step - loss: 12.4916 - mean_squared_error: 12.4916 - val_loss: 14.0421 - val_mean_squared_error: 14.0421
Epoch 94/100
11/11 [==============================] - 1s 102ms/step - loss: 13.1662 - mean_squared_error: 13.1662 - val_loss: 13.5997 - val_mean_squared_error: 

In [22]:
#``lstm2`` from the tf_models module
model_lstm2= lstm2(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_lstm2.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm2.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_lstm2 = model_lstm2.evaluate(val_dataset)
# update the result
results.update({'lstm2': eval_lstm2})

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 lstm_2 (LSTM)               (None, 100, 100)          59600     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_12 (Dense)            (None, 1)                 101       
                                                                 
Total params: 140,101
Trainable params: 140,101
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 5s 216ms/step - loss: 76.5697 - mean_squared_error: 76.5697
Epoch 2/100
11/11 [==============================] - 2s 199ms/step - los

11/11 [==============================] - 2s 212ms/step - loss: 12.7635 - mean_squared_error: 12.7635
Epoch 65/100
11/11 [==============================] - 2s 212ms/step - loss: 12.4033 - mean_squared_error: 12.4033
Epoch 66/100
11/11 [==============================] - 2s 209ms/step - loss: 12.3263 - mean_squared_error: 12.3263
Epoch 67/100
11/11 [==============================] - 2s 206ms/step - loss: 12.2536 - mean_squared_error: 12.2536
Epoch 68/100
11/11 [==============================] - 2s 209ms/step - loss: 12.5900 - mean_squared_error: 12.5900
Epoch 69/100
11/11 [==============================] - 2s 206ms/step - loss: 12.6163 - mean_squared_error: 12.6163
Epoch 70/100
11/11 [==============================] - 2s 209ms/step - loss: 12.1037 - mean_squared_error: 12.1037
Epoch 71/100
11/11 [==============================] - 2s 198ms/step - loss: 12.8043 - mean_squared_error: 12.8043
Epoch 72/100
11/11 [==============================] - 2s 198ms/step - loss: 12.4023 - mean_squared_er

In [23]:
#``blstm1`` from the tf_models module
model_blstm1= blstm1(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_blstm1.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_blstm1.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_blstm1 = model_blstm1.evaluate(val_dataset)
# update the result
results.update({'blstm1': eval_blstm1})

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 bidirectional (Bidirectiona  (None, 200)              119200    
 l)                                                              
                                                                 
 dense_13 (Dense)            (None, 1)                 201       
                                                                 
Total params: 119,401
Trainable params: 119,401
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 5s 151ms/step - loss: 85.0848 - mean_squared_error: 85.0848
Epoch 2/100
11/11 [==============================] - 2s 157ms/step - loss: 17.9643 - mean_squared_error: 17.9643
Epoch 3/100
11/11 [======

11/11 [==============================] - 2s 167ms/step - loss: 12.5342 - mean_squared_error: 12.5342
Epoch 66/100
11/11 [==============================] - 2s 155ms/step - loss: 12.7350 - mean_squared_error: 12.7350
Epoch 67/100
11/11 [==============================] - 2s 155ms/step - loss: 12.0899 - mean_squared_error: 12.0899
Epoch 68/100
11/11 [==============================] - 2s 147ms/step - loss: 12.9083 - mean_squared_error: 12.9083
Epoch 69/100
11/11 [==============================] - 2s 138ms/step - loss: 12.5824 - mean_squared_error: 12.5824
Epoch 70/100
11/11 [==============================] - 2s 141ms/step - loss: 12.1535 - mean_squared_error: 12.1535
Epoch 71/100
11/11 [==============================] - 2s 143ms/step - loss: 12.8844 - mean_squared_error: 12.8844
Epoch 72/100
11/11 [==============================] - 2s 138ms/step - loss: 12.9489 - mean_squared_error: 12.9489
Epoch 73/100
11/11 [==============================] - 2s 141ms/step - loss: 12.5330 - mean_squared_er

In [24]:
#``blstm2`` from the tf_models module
model_blstm2= blstm2(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_blstm2.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_blstm2.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_blstm2 = model_blstm2.evaluate(val_dataset)
# update the result
results.update({'blstm2': eval_blstm2})

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 100, 200)         119200    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dense_14 (Dense)            (None, 1)                 201       
                                                                 
Total params: 360,201
Trainable params: 360,201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 

11/11 [==============================] - 4s 370ms/step - loss: 12.6665 - mean_squared_error: 12.6665
Epoch 64/100
11/11 [==============================] - 4s 371ms/step - loss: 12.9866 - mean_squared_error: 12.9866
Epoch 65/100
11/11 [==============================] - 4s 367ms/step - loss: 12.7306 - mean_squared_error: 12.7306
Epoch 66/100
11/11 [==============================] - 4s 380ms/step - loss: 13.0082 - mean_squared_error: 13.0082
Epoch 67/100
11/11 [==============================] - 4s 376ms/step - loss: 12.8234 - mean_squared_error: 12.8234
Epoch 68/100
11/11 [==============================] - 4s 372ms/step - loss: 12.9354 - mean_squared_error: 12.9354
Epoch 69/100
11/11 [==============================] - 4s 368ms/step - loss: 12.6234 - mean_squared_error: 12.6234
Epoch 70/100
11/11 [==============================] - 4s 369ms/step - loss: 13.0339 - mean_squared_error: 13.0339
Epoch 71/100
11/11 [==============================] - 4s 368ms/step - loss: 12.7105 - mean_squared_er

In [25]:
#``lstm_fcn`` from the tf_models module
model_lstm_fcn= lstm_fcn(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_lstm_fcn.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm_fcn.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_lstm_fcn = model_lstm_fcn.evaluate(val_dataset)
# update the result
results.update({'lstm_fcn': eval_lstm_fcn})

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 100, 48)]    0           []                               
                                                                                                  
 conv1d_14 (Conv1D)             (None, 100, 128)     49280       ['input_9[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 100, 128)    512         ['conv1d_14[0][0]']              
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 100, 128)     0           ['batch_normalization[0][0]

11/11 [==============================] - 2s 178ms/step - loss: 4.2655 - mean_squared_error: 4.2655
Epoch 36/100
11/11 [==============================] - 2s 181ms/step - loss: 4.3955 - mean_squared_error: 4.3955
Epoch 37/100
11/11 [==============================] - 2s 181ms/step - loss: 4.2224 - mean_squared_error: 4.2224
Epoch 38/100
11/11 [==============================] - 2s 181ms/step - loss: 4.1154 - mean_squared_error: 4.1154
Epoch 39/100
11/11 [==============================] - 2s 180ms/step - loss: 3.9899 - mean_squared_error: 3.9899
Epoch 40/100
11/11 [==============================] - 2s 182ms/step - loss: 4.0246 - mean_squared_error: 4.0246
Epoch 41/100
11/11 [==============================] - 2s 183ms/step - loss: 3.6617 - mean_squared_error: 3.6617
Epoch 42/100
11/11 [==============================] - 2s 176ms/step - loss: 3.5497 - mean_squared_error: 3.5497
Epoch 43/100
11/11 [==============================] - 2s 180ms/step - loss: 3.4600 - mean_squared_error: 3.4600
Epoch

3/3 [==============================] - 1s 39ms/step - loss: 3.1280 - mean_squared_error: 3.1280


In [26]:
#``cnn_resnet`` from the tf_models module
model_cnn_resnet= cnn_resnet(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_cnn_resnet.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cnn_resnet.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_cnn_resnet = model_cnn_resnet.evaluate(val_dataset)
# update the result
results.update({'cnn_resnet': eval_cnn_resnet})

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 100, 48)]    0           []                               
                                                                                                  
 conv1d_17 (Conv1D)             (None, 100, 64)      24640       ['input_10[0][0]']               
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 100, 64)     256         ['conv1d_17[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 activation_3 (Activation)      (None, 100, 64)      0           ['batch_normalization_3[0][

11/11 [==============================] - 6s 316ms/step - loss: 71.6157 - mean_squared_error: 71.6157
Epoch 2/100
11/11 [==============================] - 3s 303ms/step - loss: 32.7377 - mean_squared_error: 32.7377
Epoch 3/100
11/11 [==============================] - 3s 287ms/step - loss: 18.2737 - mean_squared_error: 18.2737
Epoch 4/100
11/11 [==============================] - 4s 328ms/step - loss: 11.8264 - mean_squared_error: 11.8264
Epoch 5/100
11/11 [==============================] - 4s 329ms/step - loss: 9.0851 - mean_squared_error: 9.0851
Epoch 6/100
11/11 [==============================] - 3s 310ms/step - loss: 7.5763 - mean_squared_error: 7.5763
Epoch 7/100
11/11 [==============================] - 3s 312ms/step - loss: 6.3406 - mean_squared_error: 6.3406
Epoch 8/100
11/11 [==============================] - 3s 312ms/step - loss: 5.1396 - mean_squared_error: 5.1396
Epoch 9/100
11/11 [==============================] - 3s 291ms/step - loss: 5.0175 - mean_squared_error: 5.0175
Epoch

11/11 [==============================] - 4s 332ms/step - loss: 0.5051 - mean_squared_error: 0.5051
Epoch 75/100
11/11 [==============================] - 4s 327ms/step - loss: 0.5135 - mean_squared_error: 0.5135
Epoch 76/100
11/11 [==============================] - 4s 325ms/step - loss: 0.4347 - mean_squared_error: 0.4347
Epoch 77/100
11/11 [==============================] - 4s 316ms/step - loss: 0.4489 - mean_squared_error: 0.4489
Epoch 78/100
11/11 [==============================] - 4s 330ms/step - loss: 0.5397 - mean_squared_error: 0.5397
Epoch 79/100
11/11 [==============================] - 4s 324ms/step - loss: 0.6654 - mean_squared_error: 0.6654
Epoch 80/100
11/11 [==============================] - 4s 337ms/step - loss: 0.4950 - mean_squared_error: 0.4950
Epoch 81/100
11/11 [==============================] - 4s 331ms/step - loss: 0.4059 - mean_squared_error: 0.4059
Epoch 82/100
11/11 [==============================] - 4s 333ms/step - loss: 0.3460 - mean_squared_error: 0.3460
Epoch

In [36]:
results

{'cnn_lenet': [7.450009346008301, 7.450009346008301],
 'mlp4': [10.586148262023926, 10.586148262023926],
 'cnn_vgg': [2.2383792400360107, 2.2383792400360107],
 'lstm1v0': [13.068747520446777, 13.068747520446777],
 'lstm1': [11.676253318786621, 11.676253318786621],
 'lstm2': [11.668364524841309, 11.668364524841309],
 'blstm1': [11.877218246459961, 11.877218246459961],
 'blstm2': [12.565978050231934, 12.565978050231934],
 'lstm_fcn': [3.1280064582824707, 3.1280064582824707],
 'cnn_resnet': [2.0490708351135254, 2.0490708351135254]}